# Training Session
This section tries to do the following tasks:
* Train a classification model on your system
* Convert the Trained model into Keras
* Convert the Trained model into ONNX

#### Import required libraries

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
import cv2
import imghdr

#### Check data

In [ ]:
data_dir = "./data"
image_exts = ['jpg', 'jpeg', 'bmp', 'png']

#### The following Cell detect images with wrong extentions. These images could make problems during training the network. 

There is a file created by Mac called .DS_Store. Next cell will check each file and ignore it whenever it sees it.

In [ ]:
# Go to the data directory
for image_class in os.listdir(data_dir): 
    if image_class == '.DS_Store':
        continue
    # Enter images directories
    for image in os.listdir(os.path.join(data_dir, image_class)):
        if image == '.DS_Store':
            continue
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

#### Next step is to load data for training

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

#### Scale and Split Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))
data.as_numpy_iterator().next()
train_size = int(len(data)*.5)
val_size = int(len(data)*.3)
test_size = int(len(data)*.2)
print("Train size", train_size)
print("Validation", val_size)
print("Test size", test_size)
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)


#### Build deep learning model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras import activations

model = Sequential()
model.add(Conv2D(16, (3,3), input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3)))
model.add(BatchNormalization())
model.add(Activation(activations.relu))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
# model.summary()

#### Training

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')
hist = model.fit(train, epochs=300, validation_data=val, callbacks=[tensorboard_callback])

#### Plot performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

#### Evaluation

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

#### Save and Inference the Model in Keras format

In [ ]:
# Save trained model in keras (.h5)
from tensorflow.keras.models import load_model
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
# Inference the Keras model
new_model = load_model('imageclassifier.h5')
new_model.predict(np.expand_dims(resize/255, 0))

#### Save the Model in PB format and Convert it into ONNX

In [ ]:
# Save model in the saved_model format
SAVED_MODEL_DIR="./models/native_saved_model/"
model.save(SAVED_MODEL_DIR)

In [ ]:
# Convert trained model into ONNX
!python -m tf2onnx.convert --saved-model "./models/native_saved_model" --output model.onnx --opset 11 --verbose